In [34]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import time




In [83]:

# np.random.seed(1234)
stream_size = 1000
noise_var = 0.00
hyperplane_dimension = 10
drift_probability = 0.00

stream = []
# initialize and normalize w
w = np.random.normal(0,scale=1,size=hyperplane_dimension)
w = w / np.linalg.norm(w)
for k in range(stream_size):
    if np.random.uniform() < drift_probability and drift_probability != -1:
        w = np.random.normal(0, scale=1, size=hyperplane_dimension)
        w = w / np.linalg.norm(w)
    if drift_probability == -1 and k == int(stream_size/2):
        w = w / np.linalg.norm(w)
        w = np.random.normal(0, scale=1, size=hyperplane_dimension)
    # draw uniform random samples 
    X = np.random.uniform(-10, 10, hyperplane_dimension)
    # create the target parameter using the features
    y = np.dot(X,w)
    # make the stream noisy
    y = y + np.random.normal(0, noise_var)
    stream.append([X, y, w])

In [177]:
# train a RF regressor on the stream using sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

X = np.array([x[0] for x in stream])
y = np.array([x[1] for x in stream])

# normalize the data
X_ = (X - X.mean(axis=0)) / X.std(axis=0)
y_ = (y - y.mean()) / y.std()

X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.33, random_state=42)

y_test = y_test * y.std() + y.mean()

rf = RandomForestRegressor(n_estimators=10, 
                        #    max_depth=10, 
                           n_jobs=-1, 
                        #    oob_score=True,
                        #    bootstrap=True,
                           )

%time rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
# scale back the predictions
y_pred = y_pred * y.std() + y.mean()
print('RF: ', mean_absolute_error(y_test, y_pred))


# nn = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=0)
# %time nn.fit(X_train, y_train)
# y_pred = nn.predict(X_test)
# y_pred = y_pred * y.std() + y.mean()
# print('NN:', mean_absolute_error(y_test, y_pred))


# dt = DecisionTreeRegressor(max_depth=5)
# %time dt.fit(X_train, y_train)
# y_pred = dt.predict(X_test)
# y_pred = y_pred * y.std() + y.mean()
# print('DT:', mean_absolute_error(y_test, y_pred))

# svr = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
# %time svr.fit(X_train, y_train) 
# y_pred = svr.predict(X_test)
# y_pred = y_pred * y.std() + y.mean()
# print('SVR:', mean_absolute_error(y_test, y_pred))

# gpr = GaussianProcessRegressor(kernel='RBF')
# %time gpr.fit(X_train, y_train)
# y_pred = gpr.predict(X_test)
# y_pred = y_pred * y.std() + y.mean()
# print('GPR:', mean_absolute_error(y_test, y_pred))

print('std y values:', np.std(y))


CPU times: user 61.4 ms, sys: 4.11 ms, total: 65.5 ms
Wall time: 31.1 ms
RF:  1.8819798062261166
std y values: 5.618832171280289


In [44]:
print('std y values:', np.std(y))

std y values: 5.803325520336112
